## 1 Load the Merged Dataset

In [11]:
import pandas as pd
import numpy as np
import warnings

# Suppress DtypeWarnings for a cleaner output
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# Define a dictionary with corrected explicit data types
dtype_dict = {
    'user': 'category',
    'date': 'object',
    'activity': 'category',
    'pc': 'category',
    'id': 'category',
    'file_name': 'category',
    'action': 'category',
    'from': 'category',
    'to': 'category',
    'cc': 'category',
    'bcc': 'category',
    'size': 'float32',
    'device_id': 'category',
    'logon_off': 'category',
}

# --- Load the large dataset in smaller chunks to avoid MemoryError ---
chunk_size = 10000  # Reduced chunk size
chunk_list = []

print("Loading merged dataset in chunks...")
for chunk in pd.read_csv('../outputs/merged_sessions.csv', chunksize=chunk_size, dtype=dtype_dict):
    # Convert 'date' column in each chunk
    chunk['date'] = pd.to_datetime(chunk['date'])
    chunk_list.append(chunk)

# Concatenate all chunks into a single DataFrame
merged_sessions = pd.concat(chunk_list, ignore_index=True)

print("Merged dataset loaded successfully.")
print("DataFrame shape:", merged_sessions.shape)
merged_sessions.head()

Loading merged dataset in chunks...
Merged dataset loaded successfully.
DataFrame shape: (17701957, 22)


,user,date,id_x,pc_x,activity_x,id_y,pc_y,filename,activity_y,to_removable_media,...,id,pc,to,cc,bcc,from,activity,size,attachments,content_y
0,DNS1758,2010-01-02 02:19:18,{F3X8-Y2GT43DR-4906OHBL},PC-0414,Logon,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0
1,DNS1758,2010-01-02 02:31:12,{B4Q0-D0GM24KN-3704MAII},PC-0414,Logoff,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0
2,DNS1758,2010-01-02 02:34:02,{T7J1-D4HK34KV-5476TCIJ},PC-5313,Logon,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0
3,DNS1758,2010-01-02 02:53:30,{S4Y6-D8MQ05SA-0759HLIS},PC-5313,Logoff,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0
4,DNS1758,2010-01-02 04:07:31,{F3P0-E7FH78CV-4874FRGZ},PC-0012,Logon,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0


## 2 Create 'After-hours Logins' Feature

In [12]:
# Ensure the 'date' column is in datetime format
merged_sessions['date'] = pd.to_datetime(merged_sessions['date'])

# Extract the hour from the date
merged_sessions['hour'] = merged_sessions['date'].dt.hour

# Create a new feature that is 1 if the login is 'after-hours', and 0 otherwise
after_hours_start = 20  # 8 PM
after_hours_end = 6   # 6 AM

merged_sessions['after_hours_login'] = merged_sessions['hour'].apply(
    lambda h: 1 if (h >= after_hours_start or h < after_hours_end) else 0
)

print("'After-hours login' feature created.")
merged_sessions[['date', 'hour', 'after_hours_login']].head()

'After-hours login' feature created.


,date,hour,after_hours_login
0,2010-01-02 02:19:18,2,1
1,2010-01-02 02:31:12,2,1
2,2010-01-02 02:34:02,2,1
3,2010-01-02 02:53:30,2,1
4,2010-01-02 04:07:31,4,1


## 3 Quantify Activity Counts per User

In [14]:
# Print the list of columns in your merged DataFrame
print(merged_sessions.columns)

Index(['user', 'date', 'id_x', 'pc_x', 'activity_x', 'id_y', 'pc_y',
       'filename', 'activity_y', 'to_removable_media', 'from_removable_media',
       'content_x', 'id', 'pc', 'to', 'cc', 'bcc', 'from', 'activity', 'size',
       'attachments', 'content_y', 'hour', 'after_hours_login'],
      dtype='object')


In [15]:
# Group by user and date to count the number of activities
activity_counts = merged_sessions.groupby(['user', 'date']).agg(
    num_emails_sent=('from', 'count'),
    num_files_accessed=('filename', 'count'),
    num_device_events=('id_y', 'count')
).reset_index()

# Add a feature for total activity count (e.g., sum of all counts)
activity_counts['total_activity'] = (
    activity_counts['num_emails_sent'] +
    activity_counts['num_files_accessed'] +
    activity_counts['num_device_events']
)

print("Activity counts per user and day created.")
activity_counts.head()

Activity counts per user and day created.


,user,date,num_emails_sent,num_files_accessed,num_device_events,total_activity
0,AAB0162,2010-01-04 07:41:00,1,1,1,3
1,AAB0162,2010-01-04 08:34:26,1,1,1,3
2,AAB0162,2010-01-04 08:40:32,1,1,1,3
3,AAB0162,2010-01-04 08:41:06,1,1,1,3
4,AAB0162,2010-01-04 08:42:09,1,1,1,3
